In [8]:

import numpy as np
import cv2
from mss import mss
from PIL import Image

import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from dataset import get_transforms

import pyautogui
import time
import win32api, win32con

DEVICE = 'cuda'
LOAD_MODEL_FILE = "FRCNN_halo_FRCNN_model.pth"


In [9]:
CHECKPOINT_NAME = LOAD_MODEL_FILE
checkpoint = torch.load(CHECKPOINT_NAME)

transform = get_transforms(train=False)

# load a pre-trained pre-trained FRCNN from torch
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one using our one enemy class.  Use 2 here because we need a background class
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)

model.load_state_dict(checkpoint['model_state_dict'])
epoch = checkpoint['epoch']
print(f'loaded checkpoint at epoch {epoch}')

model.eval()
model.to(DEVICE)

loaded checkpoint at epoch 26


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [10]:
def apply_nms(orig_prediction, iou_thresh=0.3):

    # torchvision returns the indices of the bboxes to keep
    keep = torchvision.ops.nms(orig_prediction['boxes'], orig_prediction['scores'], iou_thresh)

    final_prediction = orig_prediction
    final_prediction['boxes'] = final_prediction['boxes'][keep]
    final_prediction['scores'] = final_prediction['scores'][keep]
    final_prediction['labels'] = final_prediction['labels'][keep]

    return final_prediction

# function to convert a torchtensor back to PIL image
def torch_to_pil(img):
    return torchvision.transforms.ToPILImage()(img).convert('RGB')

import tkinter
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')
import matplotlib.patches as patches

def plot_img_bbox(img, target):
    # plot the image and bboxes
    # Bounding boxes are defined as follows: x-min y-min width height
    fig, a = plt.subplots(1,1)
    fig.set_size_inches(5,5)
    a.imshow(img)
    for box in (target['boxes']):
        x, y, width, height  = box[0], box[1], box[2]-box[0], box[3]-box[1]
        rect = patches.Rectangle((x, y),
                                 width, height,
                                 linewidth = 2,
                                 edgecolor = 'r',
                                 facecolor = 'none')

        # Draw the bounding box on top of the image
        a.add_patch(rect)
    plt.show()


In [87]:
model.rpn.score_thresh = .5
model.rpn._pre_nms_top_n = {'training': 500, 'testing': 500}
model.rpn._post_nms_top_n = {'training': 250, 'testing': 250}

In [88]:
show = True
dbg = True

from YOLOmodel import utils

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

#centering is how much of the screen to account for, from the center
centering = 1
threshold = .8
sct = mss()

movement_scale = 1.8

w, h = pyautogui.size()

x_center = w/2
y_center = h/2

top = int(y_center - (y_center * centering))
left = int(x_center - (x_center * centering))
bottom = int(y_center + (y_center * centering))
right = int(x_center + (x_center * centering))

width = int(w * centering)
height = int(h * centering)
monitor = {'top': top, 'left': left, 'width': width, 'height': height}
resize = (448, 448)

model.to(DEVICE)

import pyautogui

while True:
    if dbg:
        start = time.time()

    orig_img = sct.grab(monitor)
    img = Image.frombytes('RGB', (orig_img.size.width, orig_img.size.height), orig_img.rgb)
    img = np.array(img)


    img = np.array(img).transpose(2,0,1)

    img = torch.tensor(img).to(DEVICE)

    img = img / 255.0

    with torch.no_grad():
        prediction = model([img.to(DEVICE)])[0]

    cpu_pred = {}
    for key, value in prediction.items():
        cpu_pred[key] = value.cpu()

    img = np.array(torch_to_pil(img))

    if dbg:
        framerate = 1/(time.time() - start)

    #add framerate to the cv2 image
    cv2.putText(img, str(framerate), (50, 50), cv2.FONT_HERSHEY_SIMPLEX,
                   1, (255, 0, 0), 2, cv2.LINE_AA)
    for i, box in enumerate(cpu_pred['boxes']):
        if cpu_pred['scores'][i] > threshold:
            box_dim = (box[3] - box[1])  #also used to scale inputs later on
            x = (box[2] + box[0]) / 2
            y = ((box[3] + box[1]) / 2) - ((box[3] - box[1]) / 3)
            cv2.circle(img,
                (int(x), int(y)),
                radius=10,
                color=(255, 0, 0),
                thickness=5
            )

            win32api.mouse_event(win32con.MOUSEEVENTF_MOVE, int(movement_scale*box_dim/orig_img.size.height*(x.item()-x_center)), int(movement_scale*box_dim/orig_img.size.height*(y.item()-y_center)))

    if show:
        cv2.imshow('test', cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR))

        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break

    else:
        print(framerate)

In [ ]:

# # to view test images
#
# import glob
# # for img in glob.glob('C:/Users/blain/Documents\Git\AutoAim\model\halo_data/*.png'):
# for img in glob.glob('C:/Users/blain\Documents\Git\AutoAim\model\halo_data/new/*'):
#
#     img = Image.open(img)
#     img = img.resize(resize)
#     img_transform = transform(img).unsqueeze(0).cuda()
#     img = np.array(img)
#
#
#     boxes = utils.cellboxes_to_boxes(model(img_transform))
#     boxes = utils.non_max_suppression(boxes[0], iou_threshold=0.5, threshold=0.4, box_format="midpoint")
#     primary, secondary = utils.pick_targets(img, boxes)
#
#     # add aim points to image
#     for aimpoint in primary:
#         print(aimpoint)
#         cv2.circle(img,
#             (int(aimpoint[0]), int(aimpoint[1])),
#             radius=10,
#             color=(255, 0, 0),
#             thickness=5
#         )
#     cv2.imshow('test', cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR))
#
#     if cv2.waitKey(0) & 0xFF == ord('q'):
#
#         break
# cv2.destroyAllWindows()